### Stock Market Data Cleaning Pipeline  

Pipeline de limpeza e preparação de dados para análise financeira do dataset `stock_part1.csv`.

---

#### **Principais Etapas**  

1. **Datas**  
   - Conversão para formato `datetime` com reconhecimento automático de padrões.  
   - Identificação de 1.015% de datas inválidas (convertidas para `NaT`).  
   - Correção de 498 registros com datas futuras (troca de dia/mês).  

2. **Tickers**  
   - Padronização em caixa alta e remoção de espaços.  
   - Correção de 2,408 tickers com erros via *fuzzy matching* (limiar: 65% de similaridade).  
   - 164 tickers raros válidos mantidos sem alteração.  

3. **Colunas Numéricas**  
   - Remoção de caracteres especiais (`$`, `Z→2`, `,`→`.`).  
   - Conversão para `float` em: `Open`, `High`, `Low`, `Close`, `Adj Close`, `Dividend`.  
   - Reordenação automática de valores para garantir `High ≥ Low`.  

4. **Volume**  
   - Padronização de unidades (`K`→×10³, `M`→×10⁶, `B`→×10⁹).  
   - Conversão para inteiro (`int64`).  

5. **Split Ratio**  
   - Identificação de 25.76% de valores inválidos.  
   - Padronização para formatos `1:1`, `2:1`, `3:2` quando possível.
   - "Valor inválidos" para aqueles que não fui possível fazer a inferencia de valor

In [ ]:
%pip install fuzzywuzzy
%pip install Levenshtein
%pip install yahoo-finance
%pip install yfinance
%pip install -U nbformat
%pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


**Importando as bibliotecas necessárias**
- pandas: manipulação do dataframe
- numpy: formulas matemáticas mais complexas
- fuzzywuzzy: quantificar relação entre palavras
- plotly.express: vizualisação dos dados
- functools:

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import plotly.express as px
from functools import lru_cache

In [ ]:
df1 = pd.read_csv('./stock_part1.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           200000 non-null  int64 
 1   Date         200000 non-null  object
 2   Ticker       200000 non-null  object
 3   Open         200000 non-null  object
 4   High         200000 non-null  object
 5   Low          200000 non-null  object
 6   Close        200000 non-null  object
 7   Adj Close    200000 non-null  object
 8   Volume       200000 non-null  object
 9   Dividend     200000 non-null  object
 10  Split Ratio  161520 non-null  object
dtypes: int64(1), object(10)
memory usage: 16.8+ MB


1. **Datas**  

Nesse bloco de código, as datas mixadas foram transformadas em um valor padrão.
Além disso, foi criada uma coluna de apoio, chamada "original date".
Datas que não puderam ser padronizadas foram contadas para um tratamento futuro.

In [ ]:
df1['original_date'] = df1['Date']
df1['Date'] = pd.to_datetime(df1['Date'], format="mixed", errors='coerce')
invalid_dates = df1[df1['Date'].isna()]
error_count = len(invalid_dates)
error_percent = error_count / len(df1["Date"])
print(f"Percentage of errors: {error_percent}%")

Percentage of errors: 1.015%


Teste para encontrar datas que ficaram no futuro

In [ ]:
from datetime import datetime

future_dates = df1[df1['Date'] > datetime.now()]["Date"]
print(future_dates)

715      2025-11-02
948      2025-09-05
1036     2025-07-03
2830     2025-09-05
2880     2025-10-01
            ...    
197447   2025-09-03
197677   2025-10-03
198021   2025-09-05
198338   2025-09-05
199004   2025-08-04
Name: Date, Length: 498, dtype: datetime64[ns]


Entendendo o comportamento das datas futuras, fica evidente que os meses e dias estão trocados

In [ ]:
maiormes = 0
for i in future_dates.dt.strftime('%Y-%m-%d'):
    diaatual = int(i[5:7])
    mesatual = int(i[-2:])
    maiordia = ( diaatual if diaatual > diaatual else diaatual)
    maiormes = ( mesatual if mesatual > maiormes else maiormes)

print(maiormes)

5


Transformação das datas futuras, remoção do timezone (informação que não tem no df e original_date foi eliminado da coluna

In [ ]:
future_dates = future_dates.apply(lambda x: x.replace(day=x.month, month=x.day) if pd.notnull(x) else x)

df1.loc[future_dates.index, "Date"] = future_dates
df1["Date"] = df1["Date"].apply(lambda x: x.replace(tzinfo=None))
# troca datas invalidas para "data inválida"
df1.loc[invalid_dates.index, "Date"] = pd.NaT

df1.drop(columns=["original_date"], inplace=True)

2. **Tickers**  
inicio da limpeza e padronização de tickers

In [ ]:
ticker_counts = df1['Ticker'].value_counts()
print(ticker_counts.describe())

count    3307.000000
mean       60.477774
std       551.331473
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max      6161.000000
Name: count, dtype: float64


Padronização básica de tickers, importação do dataframe que contem milhares de tickers válidos da NASDAQ.
Tickers válidos serão considerados aqueles que o numero de aparições é maior que 10.

In [ ]:
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()

ticker_counts = df1["Ticker"].value_counts()

valid_tickers = list(set(
    ticker_counts[ticker_counts > 10].index.tolist()
))
valid_tickers = [t.upper() for t in valid_tickers]
tickers = pd.read_csv("./tickers_new.csv")

Função que checa se o ticker é válido ou não

In [ ]:
@lru_cache(maxsize=None)
def is_valid_ticker(ticker):
    if ticker in tickers["Symbol"].values:

        return True
    return False

Função que quantifica a semelhança de um ticker comparado a cada elemento de uma lista.
Caso o nivel de semelhança seja maior que 65 ele é trocado

In [ ]:
def get_fuzzy_match(ticker, candidates, min_score=65):
    if not ticker or len(ticker) < 2:
        return None
    candidates_filtered = [t for t in candidates if abs(len(t) - len(ticker)) <= 1]
    if not candidates_filtered:
        return None
    match, score = process.extractOne(ticker, candidates_filtered)
    if score >= min_score:
        return match
    return None

Cria-se um dicionário das correções feitas, passando primeiro pela função is_valid_ticker, caso retorne falso, entra em get_fuzzy_match.

In [ ]:
corrections = {}
rare_valid = 0
for ticker in ticker_counts[ticker_counts <= 10].index:
    if not is_valid_ticker(ticker):
        corrected = get_fuzzy_match(ticker, valid_tickers)
        if corrected:
            corrections[ticker] = corrected
    else:
        rare_valid += 1

df1["Ticker"] = df1["Ticker"].replace(corrections)

print("Correções feitas:")
print(corrections)

Correções feitas:
{'RBM': 'CRM', 'BYPL': 'PYPL', 'SAOT': 'SPOT', 'BMZN': 'AMZN', 'INTB': 'INTU', 'AMZB': 'AMZN', 'DOCG': 'DOCU', 'IDM': 'IBM', 'IAPL': 'AAPL', 'ADFE': 'ADBE', 'AFZN': 'AMZN', 'AMW': 'AMD', 'UBJR': 'UBER', 'AMZ': 'AMZN', 'CSRO': 'CSCO', 'IXTU': 'INTU', 'CRN': 'CRM', 'ZMW': 'ZM', 'DMD': 'AMD', 'AAPH': 'AAPL', 'IXM': 'IBM', 'GTOG': 'GOOG', 'MSRT': 'MSFT', 'XAPL': 'AAPL', 'WBM': 'IBM', 'SHHP': 'SHOP', 'OUCL': 'ORCL', 'UBEF': 'UBER', 'DPCU': 'DOCU', 'PINL': 'PINS', 'NLLX': 'NFLX', 'AQPL': 'AAPL', 'WWTR': 'TWTR', 'CSXO': 'CSCO', 'AHD': 'AMD', 'UKER': 'UBER', 'CQM': 'CRM', 'UBEP': 'UBER', 'KMW': 'VMW', 'INLC': 'INTC', 'BABT': 'BABA', 'UBEQ': 'UBER', 'IBY': 'IBM', 'VLW': 'VMW', 'BABS': 'BABA', 'BYBA': 'BABA', 'MAPL': 'AAPL', 'IZTU': 'INTU', 'BHBA': 'BABA', 'AMK': 'AMD', 'ADBM': 'ADBE', 'ZRM': 'ZM', 'SPOH': 'SPOT', 'XYPL': 'PYPL', 'YWTR': 'TWTR', 'YNAP': 'SNAP', 'CRQ': 'CRM', 'AMJ': 'AMD', 'NVJA': 'NVDA', 'NFLJ': 'NFLX', 'BABJ': 'BABA', 'JBM': 'IBM', 'PBPL': 'PYPL', 'MNTU': 'INT

Printando a quantidade de correções feitas e de tickets que estavam no df

In [ ]:
print("corrections:", len(corrections))
print("rare valid tickers:", rare_valid)

corrections: 2408
rare valid tickers: 164


3. **Colunas Numéricas**   
Análise geral das colunas numéricas, fica evidente que todas menos "Adj Close" seguem um padrão de ruído.

In [ ]:
numerics_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "Dividend"]
def get_non_numeric_characters(column):
  print(column.name)
  tot = set("")
  for index, value in column.items():
    tot = tot | set([c for c in value if not c.isdigit()])
  print(tot)

def count_comma(column):
    count = 0
    for index,value in column.items():
        if isinstance(value, str) and len(value) >= 3:
            if value[-3] in [',', '.']:
                count += 1
    print(count)

for i in numerics_columns:
  get_non_numeric_characters(df1[i])
  count_comma(df1[i])

Open
{'Z', ',', '$', '.'}
200000
High
{'Z', ',', '$', '.'}
200000
Low
{'Z', ',', '$', '.'}
200000
Close
{'Z', ',', '$', '.'}
200000
Adj Close
{'Z', ',', '$', '.'}
200000
Volume
{'K', ',', 'o', 'l', ' ', 'B', 'M', '.', 'b', 'n', 'i'}
3503
Dividend
{'Z', ',', '$', '.'}
200000


- Removendo a colunas "Volume" para tratamento individual.
- Limpando caracteres problematicos das colunas e transformando pontos em virgulas

In [ ]:
equivalents_numeric_columns = numerics_columns
equivalents_numeric_columns.remove("Volume")

def change_caracter (value):
  value = value.strip()
  value = value.strip("$")
  value = value.replace("Z", "2")
  value = value.replace(".", ",")
  return value

for i in equivalents_numeric_columns:
  df1[i] = df1[i].apply(change_caracter)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             200000 non-null  int64         
 1   Date           197970 non-null  datetime64[ns]
 2   Ticker         200000 non-null  object        
 3   Open           200000 non-null  object        
 4   High           200000 non-null  object        
 5   Low            200000 non-null  object        
 6   Close          200000 non-null  object        
 7   Adj Close      200000 non-null  object        
 8   Volume         200000 non-null  object        
 9   Dividend       200000 non-null  object        
 10  Split Ratio    161520 non-null  object        
 11  original_date  200000 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 18.3+ MB


trocando virgula por ponto quando é para representar um numero do tipo float (todas são float) e checando de a conversão foi correta.

In [ ]:
def padronize_comma(column):
    column = column.apply(
        lambda value: value[:-3] + "." + value[-2:] if len(value) >= 3 else value
    )
    return column.astype(float)

for col_name in equivalents_numeric_columns:
    df1[col_name] = padronize_comma(df1[col_name])

print(df1["Open"].dtypes)

float64


Troca de valores para "High" ser sempre o maior valor.
"Adj Close" não foi considerado para esse tratamento por em casos especiais acabar sendo maior que o "High". Isso ocorre quando tem reverse split (as ações ao inves de serem fragmentadas, são fundidas)

In [ ]:
colunas = ["Open", "High", "Low", "Close"]
def ajustar_high_low_com_troca(row, colunas=colunas):

    valores = {col: row[col] for col in colunas}


    col_maior = max(valores, key=valores.get)
    col_menor = min(valores, key=valores.get)

    valor_maior = valores[col_maior]
    valor_menor = valores[col_menor]


    if col_maior != "High":
        row[col_maior] = row["High"]
        row["High"] = valor_maior


    if col_menor != "Low":
        row[col_menor] = row["Low"]
        row["Low"] = valor_menor

    return row


df1 = df1.apply(ajustar_high_low_com_troca, axis=1)

df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio,original_date
0,1,2023-09-03,META,909.63,1380.05,258.44,595.86,61.54,"8,846,040,350",3.57,1:1,2023-09-03
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2,"Aug 28, 2024"
2,3,2022-05-10,INTU,215.00,831.86,215.00,874.05,336.19,"8,027,589,419",2.28,2:1,"May 10, 2022"
3,4,2017-12-26,NVDA,508.65,1111.85,186.39,511.49,952.63,1939.5M,4.51,3:2,2017-12-26
4,5,2022-01-20,WORK,692.20,1425.94,340.75,378.58,1493.44,8984007687,1.93,NaN,"Jan 20, 2022"
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,2019-07-27,TSLA,269.17,418.85,144.45,144.45,893.89,4184.54M,4.71,3:2,"Jul 27, 2019"
199996,199997,2021-09-13,VMW,262.78,315.78,89.13,172.09,659.42,"5,782,458,138",1.04,3:2,"Sep 13, 2021"
199997,199998,2016-08-19,AAPL,327.66,1402.05,140.90,897.77,904.11,3104296586,3.21,NaN,2016-08-19
199998,199999,2020-06-24,DOCU,285.76,739.41,141.99,529.92,1201.64,45454067,1.51,NaN,2020-06-24


Em alguns casos, os valores de "Volume" ficam com "billion" ao lado, mesmo todas elas ja estando em bilhões (ex: 2371245326 Billion). Dessa forma é necessaria a remoção desses valores frivolos.

In [ ]:
get_non_numeric_characters(df1["Volume"].apply( lambda x: x.strip()))

for index, string in df1["Volume"].items():
  for char in string:
    if char.lower() in "bilion":
      new_string = ""
      for c in string:
        if c.lower() not in "bilion":
          new_string += c
      df1.loc[index,"Volume"] = new_string

Conversão correta em valores que tem M e ou K junto aos números
Numeros como 4.282.547.185K não são multiplicados por 1**3 pois já estão na casa dos bilhões

In [ ]:
def remove_letter(value):
  value = value.replace("M", "")
  value = value.replace("K", "")
  if value.count(",") > 1:
    value = value.replace(",", "")
  else:
    value = value.replace(",", ".")

  return float(value)

def transforming_float(value): #checar se lógica está correta
  value = value.strip()
  if "m" in value.lower() or "k" in value.lower():
    if remove_letter(value) >= 10**9:
      return int(remove_letter(value))
    if remove_letter(value) >= 1**4:
      return int(remove_letter(value) * 10**3)

    return int(remove_letter(value) * 10**6)

  return int(remove_letter(value))


df1["Volume"] = df1["Volume"].apply(transforming_float)


In [ ]:
max_volume_index = df1['Volume'].idxmax()
minvol = df1["Volume"].max()
print(minvol)
print(max_volume_index)


1012863
175058


5. **Split Ratio**
Fica evidente grande quantidade de ruído presente nos valores da couluna

In [ ]:
df1['Split Ratio'] = df1['Split Ratio'].fillna('Split inválido').astype(str)  # Substitui NaN por string vazia
get_non_numeric_characters(df1["Split Ratio"])

Split Ratio
{'L', 'N', 'p', 'X', 'O', 'B', 'H', 'M', 'E', 'I', 'D', ':', 'Z', 'K', 'F', 'U', 'n', 'e', 'W', 'P', 'i', 'T', 'o', 'v', 'l', 'Y', 'd', 'C', 'G', 'á', 'S', 'Q', ' ', 'R', 't', 'J', 'A', 'V'}


Corrigindo valores com uma letra no lugar de dois pontos (ex: 2a1 ou 3K2)

In [ ]:
def valid_split(valor):
  return valor[0].isdigit() and valor[1] != ":" and valor[2].isdigit()

for i, valor in df1["Split Ratio"].items():
    if len(valor) == 3:
        if valid_split(valor):
            df1.loc[i, "Split Ratio"] = valor[0] + ":" + valor[2]

Entendendo como é a distribuição de valores válidos

In [ ]:
def right_split(val):
    val = str(val)
    return len(val) == 3 and val[0].isdigit() and val[1] == ":" and val[2].isdigit()

poss = []

for _, val in df1["Split Ratio"].items():
    if pd.isna(val):
        continue

    if right_split(val) and val not in poss:
        poss.append(val)

print("Split Ratios válidos encontrados:", poss)

# A:1


Split Ratios válidos encontrados: ['1:1', '3:2', '2:1']


Ajustando splits parciais (ex: x:2 ou 3:f)
Em vasos com o valor 1 no final foi preciso gerar aleatóriamente os valor do numerador para manter a proporção da distribução

In [ ]:
import random
def ajust_parcial_splits(val):
    if val[0] == "2":
        return val[:-1] + "1"
    if val[0] == "3":
        return val[:-1] + "2"
    if val[2] == "1":
        newv = random.choice([1, 2])
        return f"{newv}:{val[2]}"
    return "Valor inválido"

# Iterando pelas linhas da coluna 'Split Ratio'
for index, value in df1["Split Ratio"].items():
    if not value == "Valor inválido" and (not value[0].isdigit() or not value[2].isdigit()):
        df1.loc[index, "Split Ratio"] = ajust_parcial_splits(value)

Quantidade elevada de valores inválidos e aparentemetne irreculperáveis

In [ ]:

invalid_count = df1["Split Ratio"].value_counts().get("Valor inválido", 0)
print(f"Proporção de valores inválidos: {invalid_count/len(df1['Split Ratio']):.2%}")


Proporção de valores inválidos: 25.76%


### Compreendendo os valores na tabela e tratando outliers de forma eficiênte

In [ ]:
df1["Volume"].dtypes

dtype('int64')

In [ ]:
df1.info()

#printar nulos de date e split ratio:


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             200000 non-null  int64         
 1   Date           197970 non-null  datetime64[ns]
 2   Ticker         200000 non-null  object        
 3   Open           200000 non-null  float64       
 4   High           200000 non-null  float64       
 5   Low            200000 non-null  float64       
 6   Close          200000 non-null  float64       
 7   Adj Close      200000 non-null  float64       
 8   Volume         200000 non-null  int64         
 9   Dividend       200000 non-null  float64       
 10  Split Ratio    200000 non-null  object        
 11  original_date  200000 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(2), object(3)
memory usage: 18.3+ MB


In [3]:
numeric_cols = df1.select_dtypes(include=np.number).columns
print(numeric_cols)
# Itera sobre as colunas numéricas e gera um boxplot para cada uma
for col in numeric_cols:
    fig = px.boxplot(df1, y=col, title=f'Boxplot de {col}')
    fig.show()

NameError: name 'px' is not defined

In [ ]:
df1["Ticker"].describe()

fig = px.histogram(df1, x=valid_tickers, title="Distribuição de Ticker")
fig.show()